In [ ]:
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [ ]:
import pickle
with open('grafo_bipartito.pickle', 'rb') as file:
    G = pickle.load(file)

In [ ]:
import networkx as nx
G_tot = nx.Graph(G)

In [ ]:
import random
import numpy as np
from scipy.stats import pearsonr

# Definisci il numero di utenti di interesse da estrarre casualmente
num_users_to_sample = 10  # Puoi cambiare questo numero
mse_values = []  # Lista per memorizzare gli MSE

# Esegui un ciclo per estrarre casualmente gli utenti di interesse
for _ in range(num_users_to_sample):
    # Estrai casualmente un utente tra u_1 e u_162541 (assicurati che sia un utente esistente)
    random_user = f'u_{random.randint(1, 162541)}'

    # Estrai i film valutati dall'utente specificato
    all_user_movies = [movie for movie in G.neighbors(random_user)]

    # Definisci la percentuale di film da utilizzare per il calcolo delle similarità
    percent_to_use = 0.8  # 80% dei film

    # Calcola il numero di film da utilizzare e quelli da rimuovere
    num_movies_to_use = int(len(all_user_movies) * percent_to_use)
    num_movies_to_remove = len(all_user_movies) - num_movies_to_use

    # Estrai casualmente i film da utilizzare e quelli da rimuovere
    movies_to_use = random.sample(all_user_movies, num_movies_to_use)
    movies_to_remove = [movie for movie in all_user_movies if movie not in movies_to_use]

    # G diventa G_train set
    for movie in movies_to_remove:
        G.remove_edge(random_user, movie)

    # Crea una lista vuota per memorizzare le similarità con gli altri utenti
    similarities = []

    # Calcola la similarità di Pearson tra l'utente corrente e tutti gli altri utenti con almeno 30 film in comune
    for user2 in G.nodes():
        if user2 != random_user and G.has_node(user2):
            # Estrai i film in comune tra l'utente corrente e user2
            common_movies = [movie for movie in G.neighbors(random_user) if movie in G.neighbors(user2)]

            if len(common_movies) >= 30:  # Verifica se ci sono almeno 30 film in comune
                # Estrai le valutazioni dell'utente corrente e user2 per i film in comune
                ratings_user1 = [G[random_user][movie]['weight'] for movie in common_movies]
                ratings_user2 = [G[user2][movie]['weight'] for movie in common_movies]

                # Calcola la similarità di Pearson tra le valutazioni
                pearson_similarity, _ = pearsonr(ratings_user1, ratings_user2)

                # Aggiungi la similarità alla lista delle similarità
                similarities.append((user2, pearson_similarity))

    # Ordina la lista delle similarità in ordine decrescente
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Definisci il numero di utenti simili da considerare
    num_similar_users = 10

    # Crea un dizionario per memorizzare i rating predetti per l'utente corrente per gli oggetti non valutati
    predicted_ratings = {}

    # Calcola la media dei rating dell'utente corrente rispetto agli oggetti valutati
    average_rating_u = np.sum(ratings_user1) / len(ratings_user1)

    # Trova gli utenti più simili al corrente
    similar_users = [user for user, similarity in similarities[:num_similar_users]]

    # Calcola il termine di normalizzazione Σ|suv|
    normalization_factor = np.sum([abs(similarity) for user, similarity in similarities[:num_similar_users]])

    # Per ogni oggetto (film) alpha che il corrente utente non ha ancora valutato
    for alpha in set(G.nodes()) - set(G.neighbors(random_user)):

        # Inizializza il termine di predizione
        prediction = 0

        # Inizializza la somma delle similarità assolute tra il corrente utente e gli utenti simili
        total_absolute_similarity = 0

        # Per ogni utente simile v
        for v in similar_users:
            # Verifica se v ha valutato l'oggetto alpha
            if alpha in G.neighbors(v):
                # Calcola il rating medio di v rispetto agli oggetti valutati
                rated_movies_v = [movie for movie in G.neighbors(v)]
                average_rating_v = np.sum([G[v][movie]['weight'] for movie in rated_movies_v]) / len(rated_movies_v)

                # Calcola il termine di predizione per alpha utilizzando la similarità tra il corrente utente e v
                prediction += similarities[similar_users.index(v)][1] * (G[v][alpha]['weight'] - average_rating_v)

                # Aggiungi il valore assoluto della similarità tra il corrente utente e v alla somma delle similarità assolute
                total_absolute_similarity += abs(similarities[similar_users.index(v)][1])

        # Verifica se ci sono utenti simili che hanno valutato alpha
        if total_absolute_similarity > 0:
            # Calcola il rating predetto del corrente utente per alpha utilizzando il termine di normalizzazione
            predicted_rating = average_rating_u + (prediction / total_absolute_similarity)

            # Aggiungi il rating predetto al dizionario dei rating predetti
            predicted_ratings[alpha] = predicted_rating

    # Ordina gli oggetti (film) non valutati dal corrente utente in base ai rating predetti
    recommended_movies = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)



    # Ordina gli oggetti (film) non valutati dal corrente utente in base ai rating predetti
    recommended_movies = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)

    # Prendi i primi 10 film consigliati
    top_10_recommendations = recommended_movies[:10]

    # Stampa le raccomandazioni
    print(f"Le 10 raccomandazioni migliori per {random_user} sono:")
    for movie, rating in top_10_recommendations:
        print(f"Film: {movie}, Rating Predetto: {rating}")

    predicted_ratings = {alpha: min(max(rating, 1), 5) for alpha, rating in predicted_ratings.items()}

    # Calcola il Mean Squared Error (MSE) tra le predizioni e i rating reali
    true_ratings = [G_tot[random_user][movie]['weight'] for movie in movies_to_remove if movie in predicted_ratings]
    predicted_ratings = [predicted_ratings[alpha] for alpha, _ in recommended_movies if alpha in movies_to_remove]
    mse = np.mean((np.array(true_ratings) - np.array(predicted_ratings)) ** 2)

    # Aggiungi l'MSE calcolato alla lista delle MSE
    if not np.isnan(mse):
    # Aggiungi l'MSE calcolato alla lista delle MSE solo se è un valore valido
       mse_values.append(mse)


# Calcola la media degli MSE
mean_mse = np.mean(mse_values)

# Stampa l'MSE medio per gli utenti estratti casualmente
print(f"MSE Medio per {num_users_to_sample} utenti estratti casualmente: {mean_mse}")



Le 10 raccomandazioni migliori per u_8350 sono:
Film: 3429, Rating Predetto: 4.818200408997955
Film: 2925, Rating Predetto: 4.523159303882196
Film: 3030, Rating Predetto: 4.523159303882196
Film: 1248, Rating Predetto: 4.523159303882196
Film: 1256, Rating Predetto: 4.523159303882196
Film: 6783, Rating Predetto: 4.523159303882196
Film: 6954, Rating Predetto: 4.523159303882196
Film: 7156, Rating Predetto: 4.523159303882196
Film: 38886, Rating Predetto: 4.523159303882196
Film: 2010, Rating Predetto: 4.48020663404024


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coeffic

Le 10 raccomandazioni migliori per u_102798 sono:
Film: 363, Rating Predetto: 5.284700722394221
Film: 549, Rating Predetto: 5.284700722394221
Film: 670, Rating Predetto: 5.284700722394221
Film: 973, Rating Predetto: 5.284700722394221
Film: 1066, Rating Predetto: 5.284700722394221
Film: 1153, Rating Predetto: 5.284700722394221
Film: 1154, Rating Predetto: 5.284700722394221
Film: 1260, Rating Predetto: 5.284700722394221
Film: 1913, Rating Predetto: 5.284700722394221
Film: 1929, Rating Predetto: 5.284700722394221
Le 10 raccomandazioni migliori per u_115450 sono:


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Le 10 raccomandazioni migliori per u_47214 sono:


...

Le 10 raccomandazioni migliori per u_52655 sono:


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Le 10 raccomandazioni migliori per u_29028 sono:
Film: 235, Rating Predetto: 5.463539967373572
Film: 1354, Rating Predetto: 5.463539967373572
Film: 2160, Rating Predetto: 5.463539967373572
Film: 2313, Rating Predetto: 5.463539967373572
Film: 247, Rating Predetto: 5.463539967373572
Film: 1279, Rating Predetto: 5.463539967373572
Film: 26776, Rating Predetto: 5.463539967373572
Film: 60333, Rating Predetto: 5.463539967373572
Film: 1261, Rating Predetto: 5.198756086732635
Film: 86345, Rating Predetto: 5.11060606060606
Le 10 raccomandazioni migliori per u_93750 sono:
Film: 7775, Rating Predetto: 5.927966978399404
Film: 1572, Rating Predetto: 5.897412073347361
Film: 2940, Rating Predetto: 5.897412073347361
Film: 4325, Rating Predetto: 5.897412073347361
Film: 4422, Rating Predetto: 5.897412073347361
Film: 6526, Rating Predetto: 5.897412073347361
Film: 6684, Rating Predetto: 5.897412073347361
Film: 7064, Rating Predetto: 5.897412073347361
Film: 7761, Rating Predetto: 5.897412073347361
Film: 179

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coeffic

Le 10 raccomandazioni migliori per u_148338 sono:
Film: 2109, Rating Predetto: 4.872926154847105
Film: 3087, Rating Predetto: 4.872926154847105
Film: 4002, Rating Predetto: 4.872926154847105
Film: 4679, Rating Predetto: 4.872926154847105
Film: 176371, Rating Predetto: 4.872926154847105
Film: 1263, Rating Predetto: 4.679033283866865
Film: 136449, Rating Predetto: 4.679033283866865
Film: 142115, Rating Predetto: 4.679033283866865
Film: 159817, Rating Predetto: 4.679033283866865
Film: 170705, Rating Predetto: 4.679033283866865
MSE Medio per 10 utenti estratti casualmente: 1.136708355014284


In [ ]:
true_ratings

[4.5,
 4.0,
 3.5,
 3.5,
 3.5,
 4.0,
 4.0,
 4.0,
 3.0,
 3.5,
 4.5,
 4.0,
 4.5,
 3.5,
 3.5,
 3.5,
 3.5,
 3.5,
 4.5,
 4.0,
 3.5,
 4.0,
 4.0,
 4.0,
 3.5,
 4.5,
 4.0,
 3.0,
 4.5,
 2.5,
 4.0]

In [ ]:
predicted_ratings

[5.001951550234084,
 4.952449546147633,
 4.811030723478016,
 4.6811092590410395,
 4.535925384051858,
 4.450696971112136,
 4.4158962570375735,
 4.358972411704246,
 4.3086180793829,
 4.199842118047121,
 4.199842118047121,
 4.15968607055165,
 4.077371949951931,
 4.044990310492811,
 4.027721485822303,
 3.930182897643737,
 3.9239997645987255,
 3.871578609751105,
 3.785942840581833,
 3.688864227616464,
 3.6473896256504954,
 3.4353310729956124,
 3.403758820284914,
 3.374049518328639,
 3.2736849644301453,
 3.0228375811505463,
 2.6717160037002774,
 2.6127442690055878]